In [ ]:
import reproducibility

In [ ]:
# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
import pickle, os
from utils import hash2

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan
from data_funcs import raws_data, synthetic_data, plot_data, check_data, mse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict

meso_token="b40cb52cbdef43ef81329b84e8fd874f"


In [ ]:
# Change directory for data read/write
os.chdir('data')

## Retrieve Data

In [ ]:
# read test datasets
# case_data = from_json('rnn_orig.json')
filename = 'testing_dict.pickle'
with open(filename, 'rb') as handle:
    test_dict = pickle.load(handle)
    for case in test_dict:
        test_dict[case]['case'] = case
        test_dict[case]['filename'] = filename


In [ ]:
case = 'case11'    # choose a test dataset

In [ ]:
case_data = test_dict[case]
check_data(case_data)
hours=case_data['hours']
h2=case_data['h2'] 

In [ ]:
%matplotlib inline
plot_data(case_data)

In [ ]:
plot_data(case_data,hmax=h2)

In [ ]:
%matplotlib inline
if 0:
    synt_dat=synthetic_data()  # just testinh
    check_data(case_data)
    plot_data(synt_dat)

In [ ]:
# dictionary case_data has all that is needed for the run 
# keeping the name case_data for now even if it may not be raws data



## Fit Augmented KF

In [ ]:
m,Ec = mod.run_augmented_kf(case_data)  # extract from state
case_data['m']=m
case_data['Ec']=Ec
plot_data(case_data,title2='augmented KF')

In [ ]:
plot_data(case_data,hmin=0,hmax=h2,title2='augmented KF')

In [ ]:
plot_data(case_data,hmin=h2,hmax=hours,title2='augmented KF')

In [ ]:
mse_data(case_data) 


## Fit RNN Model

In [ ]:
# Set seed for reproducibility
reproducibility.set_seed()

In [ ]:
plot_data(case_data,title2='from testing_dict.pickle')

In [ ]:
plot_data(case_data,title2='from testing_dict.pickle',hmin=0,hmax=h2)

In [ ]:
plot_data(case_data,title2='from testing_dict.pickle',hmin=h2,hmax=hours)

In [ ]:
if 'm' in case_data:
    mse_data(case_data)  # just check sdolution if there
    del case_data['m']   
    print('cleanup - removed old solution m')
if 'Ec' in case_data:
    del case_data['Ec']
    print('cleanup - removed old correction Ec')

In [ ]:
verbose = False
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data(case_data,scale=False, verbose=verbose)
check_data(rnn_dat,case=0,name='rnn_dat')

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    rnn_dat['hours'],
    activation=['linear','linear'],
    hidden_units=6,
    dense_units=1,
    dense_layers=1,
    verbose = verbose
)

In [ ]:
case_data['m'] = rnn_predict(model_predict, rnn_dat, rnn_dat['hours'], verbose = verbose)
note = 'm replaced by a solution from fmda_rnn_rain'
if 'note' in case_data:
    case_data['note'] = case_data['note'] + '\n' + note
else:
    case_data['note'] = note
check_data(case_data)

In [ ]:
mse_data(case_data)

In [ ]:
plot_data(case_data,title2='with trained RNN')

In [ ]:
plot_data(case_data,title2='with trained RNN',hmin=0,hmax=h2)

In [ ]:
plot_data(case_data,title2='RNN prediction',hmin=h2,hmax=hours)

In [ ]:
# check 5
print(hash2(model_predict.get_weights()))